In [2]:
import numpy as np
import pandas as pd
import cufflinks
import json
import os
from copy import deepcopy
from datetime import datetime

import trading_gym
from trading_gym.ray.logger import calculate_tearsheet, CustomLogger, PPOTensorboard
from trading_gym.registry.gaia.v7.env import GAIAPredictorsContinuousV7
from trading_gym.ray.models import MultiLayersPerceptron

import ray
from ray import rllib
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
# from ray.rllib.models.misc import normc_initializer, get_activation_fn
from ray import cloudpickle
from ray.utils import binary_to_hex, hex_to_binary

import tensorflow as tf
import tensorflow.contrib.slim as slim

ModelCatalog.register_custom_model(MultiLayersPerceptron.__name__, MultiLayersPerceptron)
cufflinks.go_offline()
ray.init(ignore_reinit_error=True)

2019-08-19 20:32:36,620	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-19_20-32-36_619601_51295/logs.
2019-08-19 20:32:36,737	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:13827 to respond...
2019-08-19 20:32:36,865	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:63484 to respond...
2019-08-19 20:32:36,868	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2019-08-19 20:32:36,909	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-19_20-32-36_619601_51295/logs.
2019-08-19 20:32:36,912	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-19 20:32:36,914	INFO services.py:1475 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:13827',
 'object_store_address': '/tmp/ray/session_2019-08-19_20-32-36_619601_51295/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-19_20-32-36_619601_51295/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-19_20-32-36_619601_51295'}

In [3]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
import tensorflow as tf
import tensorflow.contrib.slim as slim


class MLP(Model):
    def _build_layers_v2(self, input_dict: dict, num_outputs: int, config: dict):
        import tensorflow.contrib.slim as slim

        with tf.name_scope("fc_net"):
            last_layer = input_dict['obs']
            activation = get_activation_fn(config.get("fcnet_activation"))
            for i, size in enumerate(config.get("fcnet_hiddens"), 1):
                last_layer = slim.fully_connected(
                    inputs=last_layer,
                    num_outputs=size,
                    weights_initializer=normc_initializer(1.0),
                    activation_fn=activation,
                    scope="fc{}".format(i),
                )
#                 We don't need any dropout at this stage
#                 last_layer = tf.layers.dropout(
#                     inputs=last_layer,
#                     rate=config['custom_options']["fcnet_dropout_rate"],
#                     training=input_dict['is_training'],
#                     name="dropout{}".format(i),
#                 )
            output = slim.fully_connected(
                inputs=last_layer,
                num_outputs=num_outputs,
                weights_initializer=normc_initializer(0.01),
                activation_fn=None,
                scope="fc_out",
            )
            return output, last_layer

ModelCatalog.register_custom_model(MLP.__name__, MLP)

ModuleNotFoundError: No module named 'ray.rllib.models.misc'

In [3]:
def cloudpickleloads(obj):
    if isinstance(obj, dict):
        try:
            return cloudpickle.loads(hex_to_binary(obj["value"]))
        except:
            for key, value in obj.items():
                if isinstance(value, dict):
                    if sorted(value) == ['_type', 'value']:
                        obj[key] = cloudpickle.loads(hex_to_binary(value["value"]))
                    else:
                        obj[key] = cloudpickleloads(value)
                elif isinstance(value, list):
                    for i, item in enumerate(value):
                        obj[key][i] = cloudpickleloads(item)
    return obj

In [4]:
# This is for the 0.8 clip (the good run)

paths = {2007: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2007/experiment_state-2019-06-24_23-35-32.json',
        2008: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2008/experiment_state-2019-06-25_00-28-20.json',
        2009: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2009/experiment_state-2019-06-25_01-21-47.json',
        2010: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2010/experiment_state-2019-06-25_02-14-52.json',
        2011: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2011/experiment_state-2019-06-25_03-08-23.json',
        2012: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2012/experiment_state-2019-06-25_04-00-54.json',
        2013: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2013/experiment_state-2019-06-25_04-54-29.json',
        2014: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2014/experiment_state-2019-06-25_05-47-18.json',
        2015: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2015/experiment_state-2019-06-25_06-40-12.json',
        2016: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2016/experiment_state-2019-06-25_07-33-17.json',
        2017: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2017/experiment_state-2019-06-25_08-26-33.json'
        }

In [5]:
policies = dict()
env_configs = dict()
for year, path in paths.items():
    print(year)
    with open(path) as f:
        metadata = json.load(f)
        checkpoints = metadata['checkpoints']
        runner_data = metadata['runner_data']
        stats = metadata['stats']

    for checkpoint in checkpoints:
        checkpoint = cloudpickleloads(checkpoint)
        cp = cloudpickle.loads(hex_to_binary(checkpoint['_checkpoint']))
        config = checkpoint['config']
        env_cls = config['env']
        env_config = config['env_config']
        path_restore = os.path.join(checkpoint['logdir'], cp.value)
        

    agent = rllib.agents.ppo.PPOTrainer(config, env_cls)
    agent.restore(path_restore)
    
    policies[year] = agent.get_policy()
    env_configs[year] = env_config

2019-06-26 09:26:56,965	WARNING ppo.py:153 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).


2007


2019-06-26 09:26:58,241	INFO policy_evaluator.py:312 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


<class 'module'>


2019-06-26 09:26:58,528	INFO dynamic_tf_policy.py:265 -- Initializing loss function with dummy input:

{ 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
  'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
  'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
  'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 2) dtype=float32>,
  'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
  'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 3) dtype=float32>,
  'obs': <tf.Tensor 'default_policy/observation:0' shape=(?, 3) dtype=float32>,
  'prev_actions': <tf.Tensor 'default_policy/action:0' shape=(?, 2) dtype=float32>,
  'prev_rewards': <tf.Tensor 'default_policy/prev_reward:0' shape=(?,) dtype=float32>,
  'rewards': <tf.Tensor 'default_policy/rewards:0' shape=(?,) dtype=float32>,
  'value_targets': <tf.Tensor 'default_policy/value_targets:0' shape=

...

/home/Nicholas/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.

/home/Nicholas/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.

2019-06-26 09:28:29,208	WARNING ppo.py:153 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).


2017
(pid=34281) 2019-06-26 09:28:33,484	INFO dynamic_tf_policy.py:265 -- Initializing loss function with dummy input:
(pid=34281) 
(pid=34281) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=34281)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=34281)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=34281)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 2) dtype=float32>,
(pid=34281)   'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
(pid=34281)   'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 3) dtype=float32>,
(pid=34281)   'obs': <tf.Tensor 'default_policy/observation:0' shape=(?, 3) dtype=float32>,
(pid=34281)   'prev_actions': <tf.Tensor 'default_policy/action:0' shape=(?, 2) dtype=float32>,
(pid=34281)   'prev_rewards': <tf.Tensor 'default_policy/prev_reward:0' shape=(?,) dtype=float32>,
(pid=34281

2019-06-26 09:28:35,095	INFO policy_evaluator.py:312 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


<class 'module'>


/home/Nicholas/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.

2019-06-26 09:28:39,614	INFO policy_evaluator.py:735 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f5b01f487f0>}
2019-06-26 09:28:39,616	INFO policy_evaluator.py:736 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f5b01f483c8>}
2019-06-26 09:28:39,617	INFO policy_evaluator.py:347 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f5b01f48208>}
2019-06-26 09:28:39,928	INFO multi_gpu_optimizer.py:80 -- LocalMultiGPUOptimizer devices ['/cpu:0']
/home/Nicholas/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. T

In [7]:
levels = list()
mappings = pd.DataFrame()
mapping_functions = dict()
for year, policy in policies.items():
    print('_______________________________________{}____________________________________________'.format(year))
    
    env_config = env_configs[year]
    env = env_cls(env_config)
    episode = env.sample_episode(
        fold='test-set',
        policy=policy,
    )
    renderer = env.render(
        benchmark=env._load_benchmark().squeeze().loc[str(year)],
        risk_free=env._load_risk_free().squeeze().loc[str(year)],
    )
    renderer.plotly_report()
    renderer.tearsheet()
    
    levels.append(renderer.level.to_frame().pct_change())
    
    
    
    actions = pd.DataFrame(episode.actions, index=episode.time, columns=env.action_space.contracts[1:])
    states = pd.DataFrame(episode.states, index=episode.time)
    target_russell = actions[actions.columns[0]]
    mapping = states[0].to_frame('GAIA Predictor').join(target_russell)
    mappings = mappings.append(mapping)
    
    
    
    this_mapping = dict()
    env.reset()
    for predictor in np.linspace(-0.01, 0.01, 500):
        from trading_gym.registry.gaia.v7.env import EventGAIAPredictor
        from scipy.special import softmax
        event = EventGAIAPredictor(time=datetime.now(), equities=predictor)
        env.add_event(event)
        state = env.state.get_values()

        _, _, info = policy.compute_single_action(state, [])
        logits = info['behaviour_logits']
        action = softmax(logits)
        target_weight_equity = action[0]
        this_mapping[predictor] = target_weight_equity
    mapping_functions[year] = this_mapping

_______________________________________2007____________________________________________


_______________________________________2008____________________________________________


_______________________________________2009____________________________________________


_______________________________________2010____________________________________________


_______________________________________2011____________________________________________


...

_______________________________________2013____________________________________________


_______________________________________2014____________________________________________


_______________________________________2015____________________________________________


_______________________________________2016____________________________________________


_______________________________________2017____________________________________________


In [13]:

mappings.sort_index().iplot(
#     secondary_y='Russell 1000'  ---    This was here before
    secondary_y=str(mappings.columns[-1]),
    vline=[str(year) for year in range(2008, 2019)]
)

In [14]:
pd.DataFrame(mapping_functions).iplot(title='Mapping functions trained with deep reinforcement learning', yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')

In [15]:
mappings.set_index('GAIA Predictor').iplot(kind='scatter', mode='markers', size=3, yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')